<a href="https://colab.research.google.com/github/cd-public/compute/blob/main/ps/PDA_CFG_CNF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Context Free Grammars and Pushdown Automata
*Theory of Computation: Problem Set Two*

## Introduction

### Pushdown Automaton (PDA)

A Pushdown Automaton (PDA) is a 6-tuple:

**(Q, Σ, Γ, δ, q₀, Z₀, F)**

Where:

* **Q:** A finite, non-empty set of *states*.

* **Σ:** A finite, non-empty set of *input symbols* (the alphabet).

* **Γ:** A finite, non-empty set of *stack symbols*.

* **δ:** A *transition function*, where δ: Q × (Σ ∪ {ε}) × Γ → 2^(Q × Γ*).  
  
    * This is the key difference from a DFA.  It takes a state, an input symbol (or ε for an epsilon-transition), and a stack symbol as input.  
    * It returns a *set* of possible next configurations, where each configuration consists of a next state and a string to replace the top stack symbol with.  
    * This allows for nondeterminism and stack manipulation.

* **q₀:** The *initial state*, where q₀ ∈ Q.

* **F:** A set of *accepting states* (or final states), where F ⊆ Q.  Acceptance can be defined in two ways:
    * **Acceptance by final state:** The PDA accepts the input string if, after processing the entire string, the PDA is in one of the accepting states (F).
    * **Acceptance by empty stack:** The PDA accepts the input string if, after processing the entire string, the stack is empty.  (Sometimes, both final state and empty stack acceptance are used, but they define the same class of languages.)

**In simpler terms:**

A PDA is like a DFA but with an added *stack*.  It reads input symbols one at a time, just like a DFA.  However, its transitions depend not only on the current state and input symbol but also on the symbol at the *top of the stack*.  The PDA can also *manipulate the stack* by pushing new symbols onto it or popping symbols off.  This stack gives the PDA more memory and allows it to recognize a wider class of languages than DFAs (specifically, context-free languages).  The non-deterministic nature of the transition function means that for a given input, the PDA may have multiple possible execution paths.

$D=\{0^k1^k|k∈\mathbb{N}\}$

In [ ]:
q_1 = lambda s, stack : [stack.append('0'), q_1(s[1:], stack)][-1] if s[0] == '0' and len(s) > 1 else [stack.pop(), q_2(s[1:], stack)][-1]
q_2 = lambda s, stack : [stack.pop(), q_2(s[1:],stack)][-1] if s[0] == '1' and len(s) > 1 else q_n(s, stack)
q_n = lambda s, stack : s == '1' and len(stack) == 1
q_1('000111', []), q_1('00111', []), q_1('00011', [])

### Context-Free Grammar (CFG)

A Context-Free Grammar (CFG) is a 4-tuple:

**(V, Σ, R, S)**

Where:

* **V:** A finite, non-empty set of *variables* (or non-terminal symbols).  These represent syntactic categories or placeholders.

* **Σ:** A finite, non-empty set of *terminals*.  These are the actual symbols that make up the strings of the language.  Σ and V are disjoint (V ∩ Σ = ∅).

* **R:** A finite set of *production rules*.  Each rule has the form A → α, where A ∈ V and α ∈ (V ∪ Σ)*.  This means the left-hand side of a rule is always a single variable, and the right-hand side can be any string of variables and terminals (including the empty string, ε).

* **S:** The *start symbol*, where S ∈ V.  This is the variable from which derivations begin.

**In simpler terms:**

A CFG is a set of rules that describe how to generate strings of a language.  You start with the start symbol (S) and repeatedly apply the production rules, replacing variables with the strings on the right-hand side of the rules.  This process continues until all variables have been replaced by terminals.  The set of all strings that can be derived in this way from the start symbol is the language generated by the CFG.  The "context-free" part means that the left-hand side of a production rule is a single variable, so the replacement can happen regardless of the surrounding symbols (the "context").

**Example:**

A simple CFG for arithmetic expressions:

* V = $\{E\}$  
  * (Expression)
* Σ = $\{+, *, (, ), a\}$
  * (plus, times, open paren, close paren, identifier)
* R =
$$
\begin{align}
\{&&\\
    &E → &E + E\\
    &E → &E \times E\\
    &E → &(E)\\
    &E → &a\\
  \}&&\\
\end{align}
$$
* S = E

This grammar can generate expressions like "$a + a \times a$" or "$(a + a) * a$".

The following isn't quite right. What's wrong?

In [ ]:
from itertools import count # infinite iterator
from itertools import combinations_with_replacement as cwr
# define variables, so we can use them
rules = (
  lambda s : s.replace("E", "E+E"),
  lambda s : s.replace("E", "ExE"),
  lambda s : s.replace("E", "(E)"),
  lambda s : s.replace("E", "a")
)
trans = lambda fs, s: fs[0](trans(fs[1:], s)) if len(fs) else s

G_2 = (trans(fs,'E') for n in count() for fs in cwr(rules, n))

next(G_2), next(G_2), next(G_2)

### Chomsky Normal Form (CNF)

A Context-Free Grammar (CFG) is in Chomsky Normal Form (CNF) if all production rules are in one of the following forms:

1. **A → BC:**  Where A, B, and C are variables (non-terminals).  Neither B nor C can be the start symbol.

2. **A → a:**  Where A is a variable and 'a' is a terminal.

3. **S → ε:** Where S is the start symbol and ε is the empty string.  This rule is only allowed if ε is in the language generated by the grammar.

**In simpler terms:**

CNF is a standardized form for CFGs that simplifies certain proofs and algorithms related to context-free languages.  It restricts the structure of production rules so that they are either:

* Replacing a variable with two other variables.
* Replacing a variable with a single terminal.
* (Possibly) replacing the start symbol with the empty string.

**Why CNF is Important:**

CNF is important because:

* **Simplifies parsing:** Many parsing algorithms (like the CYK algorithm) require the grammar to be in CNF.
* **Theoretical significance:**  It's a useful normal form for proving properties of context-free languages.  Any CFG can be converted to CNF.

**Example Conversion (Simplified):**

Let's say we have a rule `A → X Y Z`.  In CNF, we would introduce a new variable, say `V`, and rewrite the rule as:

* `A → X V`
* `V → Y Z`

This breaks down the longer production into a series of productions where each right-hand side has at most two symbols, and those symbols are variables (except for the base case of a variable deriving a terminal).  More complex transformations are needed to handle other cases, like rules with terminals on the right-hand side or rules where a variable derives itself.

## Problem 1

Construct the pushdown automata that recognizes the language of bit strings for which there are the same number of `0` characters/digits/symbols/letters and `1` characters/digits/symbols/letters.

You may use the transitions from the introduction to help you get started, but must specify a full-tuple of the form:

```py
M = (Q,S,G,d,q_0,F)
```


In [ ]:
# Your Code Here

## Problem 2

Create in the Python this simple yet unambigious CFG for arithmetic expressions:

* V = $\{E, T, F\}$  
  * (Expression, Term, Factor)
* Σ = $\{+, *, (, ), a\}$
  * (plus, times, open paren, close paren, identifier)
* R =
$$
\begin{align}
\{&&\\
    &E → &E + E,\\
    &E → &E \times E,\\
    &T → &T * F,\\
    &T → &F,\\
    &F → &(E),\\
    &F → &a\\
  \}&&\\
  \end{align}
$$
* S = E

This grammar can generate expressions like "a + a * a" or "(a + a) * a".

In [ ]:
# Your Code Here

## Problem 3

Translate your Problem 2 solution into Chomsky Normal Form (CNF)

In [ ]:
# Your Code Here

## Problem 4

Translate your Problem 2/3 solution into a Pushdown Automaton (PDA)

In [ ]:
# Your Code Here

## Problem 5

Translate your Problem 1 solution into a CNF CFG (a Chomsky Normal Form Context Free Grammar).